In [ ]:
# リスト 2.1.20
# Yahoo!ショッピング Web API を用いてカテゴリ ID 一覧を取得するための準備コード
# ------------------------------------------------------------------------------------------------
# 目的:
#   - カテゴリ検索エンドポイント `/ShoppingWebService/V1/json/categorySearch` を叩き、
#     親カテゴリIDから子カテゴリ（Children）を列挙する土台を用意する。
#   - このスニペットでは「API 呼び出し関数」と「カテゴリ抽出ジェネレータ」を定義する。
#
# 背景/注意:
#   - appid は Yahoo! デベロッパーネットワークで取得したアプリケーションIDを使用する（ダミー 'xxxx' のままでは動かない）。
#   - レート制御: `r_get` 内で 1 秒スリープしているが、正式なレート上限はドキュメントに従うこと。
#   - 例外処理: `get_cats` の `except: pass` は全例外を握りつぶすため、実運用ではログ/リトライへ改善推奨（ここでは挙動を変えない）。
#   - ネットワーク健全性: タイムアウト、リトライ、ステータスコード検査、User-Agent 指定等は本例では省略（最小例）。
#   - 文字コード: API 応答は JSON（UTF-8）想定。`requests` の `.json()` は応答の `Content-Type` を基にデコードする。
#   - 返却構造: `ResultSet` → `'0'` → `Result` → `Categories` → `Children` に子カテゴリがぶら下がる想定。
#     サービスの仕様変更や親カテゴリが末端の場合は `KeyError` が起き得る（ここでは例外を無視してスキップ）。
#
# 進展:
#   - `all_categories_file` は次のリストで CSV に保存する前提のプレースホルダ。本スニペット中では未使用。

import requests
import json
import time
import csv

# エンドポイント（JSON 版 categorySearch）
url_cat = "https://shopping.yahooapis.jp/ShoppingWebService/V1/json/categorySearch"

# アプリケーションID（p.34 の方法で取得した値を設定）
appid = "xxxx"

# すべてのカテゴリを書き出す CSV のパス（このリストでは未使用、後続で利用する想定）
all_categories_file = "./all_categories.csv"


def r_get(url, dct):
    """
    API リクエスト発行の薄いラッパ（最小実装）。

    パラメータ:
        url : str
            送信先エンドポイント。
        dct : dict
            クエリパラメータ。ここでは { 'appid': appid, 'category_id': <ID> } を想定。

    仕様/挙動:
        - レート抑制のため呼び出し毎に 1 秒スリープ。
        - 返り値は requests.Response（ステータス検査や例外処理は行わない）。
          実運用では:
            * timeout=... の指定
            * ステータスコード検査（.raise_for_status()）
            * 429/5xx への指数バックオフ
            * 固定 User-Agent の付与
          などを追加すること。
    """
    time.sleep(1)  # 単純なレート制御（1 リクエスト/秒）
    return requests.get(url, params=dct)


def get_cats(cat_id):
    """
    親カテゴリ ID を受け取り、その直下の子カテゴリを (子ID, タイトル辞書) の形で逐次返すジェネレータ。

    パラメータ:
        cat_id : str または int
            親カテゴリを表す ID。ルート（'1' 等）から辿るケースを想定。

    返却:
        yield (child_id, title_dict)
            child_id : str
                子カテゴリの ID。
            title_dict : dict
                'short' / 'medium' / 'long' をキーに持つタイトル表記群。

    実装メモ:
        - 応答 JSON の想定パス:
          ResultSet -> '0' -> Result -> Categories -> Children -> <連番キー> -> { Id, Title:{Short,Medium,Long} }
        - 仕様変更・末端カテゴリ・レスポンス異常時は例外が発生し得るため、
          現在は裸の except で握りつぶし、何も yield せずに終わる。
          本番では例外の種類ごとに:
            * ネットワーク: リトライ
            * JSON 構造: ログ/スキップ
            * 権限/パラメータ: 早期中断
          等の分岐を行うこと。
    """
    try:
        # API 叩き（appid と親 ID をクエリに付与）
        result = r_get(url_cat, {"appid": appid, "category_id": cat_id})

        # JSON にデコードし、想定の階層から子カテゴリ辞書を取り出す
        cats = result.json()["ResultSet"]["0"]["Result"]["Categories"]["Children"]

        # Children は { '0': {...}, '1': {...}, ..., '_container': ... } のような構造を取り得る
        for i, cat in cats.items():
            if i != "_container":  # メタ要素（コンテナ情報）はスキップ
                # 各子カテゴリの ID と 3 種のタイトル（短・中・長）を取り出して返す
                yield cat["Id"], {
                    "short": cat["Title"]["Short"],
                    "medium": cat["Title"]["Medium"],
                    "long": cat["Title"]["Long"],
                }
    except:
        # 例外の握りつぶし（最小例）。本番ではログやリトライに置き換えること。
        # 例: except requests.exceptions.RequestException as e: ... など
        pass

In [ ]:
# リスト 2.1.21
# カテゴリ一覧CSVファイルの生成
# ------------------------------------------------------------------------------------------------
# 目的:
#   - 既に定義済みの `get_cats(cat_id)`（親カテゴリID→直下の子カテゴリを列挙）を用いて、
#     Yahoo!ショッピングのカテゴリを「レベル1→2→3」の3層で全走査し、CSV に逐次書き出す。
#
# 前提:
#   - 直前のリスト 2.1.20 までで以下が定義済みであること:
#       * `all_categories_file` : 出力CSVのパス
#       * `get_cats(cat_id)`    : 子カテゴリ (Id, Title[Short/Medium/Long]) を yield するジェネレータ
#       * `csv` モジュールの import
#
# 出力仕様（1行あたりの列順）:
#   [カテゴリコードlv1, カテゴリコードlv2, カテゴリコードlv3,
#    カテゴリ名lv1,   カテゴリ名lv2,   カテゴリ名lv3,   カテゴリ名lv3_long]
#
# 実装メモ（挙動を変えずに背景のみ説明）:
#   - まずヘッダ行を書き出し、その後 3 重ループで L1→L2→L3 を辿り、バッファ `output_buffer` に
#     行を貯めた上で、都度 CSV に追記する（この例では L3 内側で即フラッシュ）。
#   - `get_cats` は内部でネットワーク I/O を伴い、例外を握りつぶしている（最小例）。実運用では
#     エラー分類（ネットワーク/構造/権限）に応じたリトライ・ログ化が望ましい。
#   - 本スニペットではファイルのエンコーディング指定を省略（Python 既定）。Excel 互換を重視する場合は
#     `encoding='utf-8-sig'` 等を利用するとよい（ここでは挙動を変えない）。
#   - `KeyError` の捕捉は L2 取得時に Children が存在しない等の構造差分を想定している。
# ------------------------------------------------------------------------------------------------

# ヘッダ定義（列名）
output_buffer = [
    [
        "カテゴリコードlv1",
        "カテゴリコードlv2",
        "カテゴリコードlv3",
        "カテゴリ名lv1",
        "カテゴリ名lv2",
        "カテゴリ名lv3",
        "カテゴリ名lv3_long",
    ]
]

# 既存ファイルを上書きモードで開き、まずはヘッダだけを書き出す
with open(all_categories_file, "w") as f:
    writer = csv.writer(
        f, lineterminator="\n"
    )  # 改行は \n に統一（Windows でも \r\n にならない）
    writer.writerows(output_buffer)
    output_buffer = []  # バッファをクリア（以降はデータ行を貯めて使う）

# カテゴリレベル1（ルート=1 から直下の子を列挙）
for id1, title1 in get_cats(1):
    print("カテゴリレベル１ :", title1["short"])
    try:
        # カテゴリレベル2（L1 の子を列挙）
        for id2, title2 in get_cats(id1):

            # カテゴリレベル3（L2 の子を列挙）
            for id3, title3 in get_cats(id2):
                # 1 レコード分を作成し、バッファに追加
                wk = [
                    id1,
                    id2,
                    id3,
                    title1["short"],
                    title2["short"],
                    title3["short"],
                    title3["long"],
                ]
                output_buffer.append(wk)

                # バッファを即フラッシュして CSV に追記
                # ここでは L3 の各反復ごとに書き出している（メモリ節約/クラッシュ時の消失低減）。
                with open(all_categories_file, "a") as f:
                    writer = csv.writer(f, lineterminator="\n")
                    writer.writerows(output_buffer)
                    output_buffer = []  # フラッシュ後は空に戻す

    except KeyError:
        # 例: ある L2 に Children が存在しない（末端）場合などの構造差分をスキップ
        continue

In [ ]:
# リスト 2.1.22
# CSVファイルの内容確認

import pandas as pd
from IPython.display import display

df = pd.read_csv(all_categories_file)
display(df.head())

In [ ]:
# リスト 2.1.23
# スマホのコード確認

df1 = df.query("カテゴリコードlv3 == '49331'")
display(df1)

In [ ]:
# リスト 2.1.24
# レビューコメントの取得（Yahoo!ショッピング Web API "reviewSearch" を用いた最小～実務寄りの実装に向けた詳細解説コメント付き）
# ------------------------------------------------------------------------------------------------
# 目的:
#   - Yahoo!ショッピングのレビュー検索API（/ShoppingWebService/V1/json/reviewSearch）を用いて、
#     指定したカテゴリIDに属する商品レビューをページングしながら最大 max_items 件まで収集する。
#
# 全体設計のポイント（理論・運用面の補足。挙動は変更せずコメントのみ追加）:
#   1) API スループットとレート制御:
#      - API 仕様上、1回の取得件数 'results' は最大 50。大量収集では「開始位置 'start'」を更新して複数回叩く必要がある。
#      - 本コードは呼び出しごとに time.sleep(1) を入れている（1 req/sec の単純レート制御）。正式な上限はドキュメントに従うこと。
#   2) ページングの決定性:
#      - 'start' は現在の先頭位置、'ret' は今回返却件数、'avl' はヒット総数。ループ内で start += ret とすることでスキップなく前進する。
#      - 返却件数 'ret' が 0 なら先に進まないため無限ループの危険があるが、多くのAPIでは 'ret' > 0 を保証する。守られない可能性も理論上あるので実運用ではブレーク条件を加えると良い。
#   3) エラー設計:
#      - HTTP 200 以外を "result.ok" で検出。400 は読み飛ばし、それ以外は exit(True) で中断する最小方針。
#      - 実運用では requests の例外（Timeout/ConnectionError）も捕捉し指数バックオフを適用するのが理にかなう（ここでは最小実装）。
#   4) データ整形と CSV 想定:
#      - 改行やカンマをレビュー本文から除去/置換しており（','→'、'、'\n'除去）、CSV に書き出しても列崩れしない意図。
#      - Unicode 正規化（NFC/NFKC）は下流要件に応じて追加検討。
#   5) しきい値フィルタ:
#      - 最小/最大文字数でレビュー本文をフィルタリング。短すぎるノイズや極端に長い外れ値の除外に有効。
#   6) appid と規約:
#      - appid は個人/アプリ固有の認証値。公開リポジトリでは秘匿（環境変数等）すべき。書籍指示に従い伏せ字にするのが前提。
#   7) 取得の再現性・監査:
#      - 研究/運用では、取得日時やクエリ、API バージョン、合計ヒット数などのメタを同時保存すると監査可能性が高まる。
#
# 取得データのスキーマ（results の各要素 buff のキー）:
#   - id      : そのカテゴリ取得内での通し番号（1..max_items）
#   - title   : レビュータイトル（改行削除・カンマは読点へ置換）
#   - rate    : 評点（'Ratings.Rate' を float→int）
#   - comment : レビュー本文（改行削除・カンマは読点へ置換）
#   - name    : 対象商品の名称
#   - code    : 対象商品のコード
#
# 注意:
#   - このファイル単体は「レビュー配列を返す」関数群であり、CSV 出力は別スクリプト側の責務を想定している。
#   - ここでは動作を変えないため、timeout 指定や User-Agent などは追加していない（実運用では追加推奨）。

import requests
import time

# reviewSearch エンドポイント（JSON）
url_review = "https://shopping.yahooapis.jp/ShoppingWebService/V1/json/reviewSearch"

# アプリケーションid
# 書籍では appid は伏せ字にしてください（公開リポジトリでは環境変数等で管理するのが推奨）
appid = "dj0zaiZpPUZCZFh2WjRYM1V1WCZzPWNvbnN1bWVyc2VjcmV0Jng9ZmE-"

# 1リクエストで取得するレビュー件数（API仕様の上限は 50）
num_results = 50

# 1カテゴリあたりの最大取得件数（上位呼び出し側が get_reviews(..., max_items) に渡す想定の参考値）
# ※ この変数は本スニペット内では関数引数により上書きされるため未使用だが、呼び出し側で使用される場合がある。
num_reviews_per_cat = 99999999

# レビュー本文の長さフィルタ（最小/最大）。本文が短すぎる・長すぎるものは読み飛ばす。
max_len = 10000
min_len = 50


def r_get(url, dct):
    """
    API リクエストの薄いラッパ。
    - 単純なレート制御として、呼び出しごとに 1 秒のウェイトを入れる。
    - 返り値は requests.Response のまま返す（ステータス検査は呼び出し側）。
    実運用では:
      * timeout=(接続, 読み取り) の指定
      * 再試行/指数バックオフ（429/5xx 対応）
      * 固定 User-Agent 明示
      * 例外捕捉（requests.exceptions.*）
    を追加するのが望ましい。
    """
    time.sleep(1)  # 1回で1秒あける（APIサーバに配慮したスロットリング）
    return requests.get(url, params=dct)


def get_reviews(cat_id, max_items):
    """
    指定したカテゴリIDに属するレビューを最大 max_items 件まで取得し、辞書の配列で返す。

    パラメータ:
        cat_id    : int/str
            Yahoo!ショッピングのカテゴリID
        max_items : int
            収集するレビューの最大件数（上限）

    戻り値:
        results : list[dict]
            各 dict は {id, title, rate, comment, name, code} を含む。

    動作詳細:
      - ページング:
          'start' を 1 から始め、'results'=num_results（≤50）でAPIを叩く。
          応答から 'ret'（totalResultsReturned）を読み、start += ret で次ページ先頭へ進む。
      - フィルタリング:
          本文長が [min_len, max_len] の範囲外ならスキップ。
      - 整形:
          CSV 互換性を意識してタイトル・本文の改行を除去し、カンマは読点へ置換。
      - 終了条件:
          items（有効カウント）が max_items に達したら終了。
      - エラー:
          HTTP 200 以外はステータスと理由を出力。400 は読み飛ばし（break）、その他は exit(True) で終了。
          ※ 本番環境では exit より例外送出やリトライの方が運用しやすい。
    """
    # 実際に返した件数
    items = 0
    # 結果配列
    results = []
    # 開始位置（1始まり）
    start = 1

    while items < max_items:
        # API 呼び出し（カテゴリIDとページングパラメータ）
        result = r_get(
            url_review,
            {
                "appid": appid,
                "category_id": cat_id,
                "results": num_results,
                "start": start,
            },
        )

        if result.ok:
            # 正常応答（HTTP 200 系）。ResultSet を取り出す。
            rs = result.json()["ResultSet"]
        else:
            # エラー応答。最低限の診断出力を行う。
            print(
                "エラーが返されました : [cat id] {} [reason] {}-{}".format(
                    cat_id, result.status_code, result.reason
                )
            )
            if result.status_code == 400:
                # 400 Bad Request は入力の問題が多く、継続不能なことが多いが、ここでは「中止せず読み飛ばす」方針
                print("ステータスコード400(badrequestは中止せず読み飛ばします")
                break
            else:
                # それ以外のエラーは即時終了（最小実装）。実運用では例外送出やリトライに切り替える。
                exit(True)

        # ページング関連のメタ値を取得（監査/ログに有用）
        avl = int(rs["totalResultsAvailable"])  # 総ヒット数
        pos = int(rs["firstResultPosition"])  # 今回返却の先頭位置
        ret = int(rs["totalResultsReturned"])  # 今回の返却件数（≤ num_results）

        # レビュー本体を取得（配列想定）
        reviews = result.json()["ResultSet"]["Result"]

        for rev in reviews:
            # 本文長でフィルタリング（短すぎ/長すぎはスキップ）
            desc_len = len(rev["Description"])
            if min_len > desc_len or max_len < desc_len:
                continue

            items += 1  # 有効件数としてカウント

            # 1 レビュー分の辞書を組み立て（CSV 安全性のための軽微な整形を含む）
            buff = {}
            buff["id"] = items  # 本カテゴリ取得内の通し番号

            # タイトル: 改行削除・カンマ→読点（CSV 列崩れ防止）
            buff["title"] = rev["ReviewTitle"].replace("\n", "").replace(",", "、")

            # 評点: 文字列/浮動小数の可能性があるため float を経由して int に
            buff["rate"] = int(float(rev["Ratings"]["Rate"]))

            # 本文: 同様に改行削除・カンマ→読点
            buff["comment"] = rev["Description"].replace("\n", "").replace(",", "、")

            # 商品名・商品コード（ターゲット情報）
            buff["name"] = rev["Target"]["Name"]
            buff["code"] = rev["Target"]["Code"]

            results.append(buff)

            # 目標件数に達したらループ離脱
            if items >= max_items:
                break

        # 次ページ先頭へ（今回返却件数ぶんだけ進める）
        start += ret

        # ret が 0 の場合、先に進まず無限ループの恐れがある。
        # 実運用では `if ret == 0: break` を入れるのが安全だが、ここでは元コードの挙動を尊重して追加しない。

    return results

In [ ]:
# リスト2.1.25
# コメント一覧の取得と保存

import json
import pickle

# get_reviews(code, count) レビューコメントの取得
# code: カテゴリコード (all_categories.csv)に記載のもの
# count: 何件取得するか

result = get_reviews(49331, 5)
print(json.dumps(result, indent=2, ensure_ascii=False))